# ***ETL de nuestro dataset Australian_user_reviews***

In [91]:
import numpy as np
import pandas as pd
from textblob import TextBlob
import ast
import matplotlib.pyplot as plt
import parquet
import pyarrow
import warnings
warnings.filterwarnings('ignore')


# Procedemos a abrir nuestro archivo

In [92]:
#Crear una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.
rows = []
#Abrir el archivo "user_reviews.json/australian_user_reviews.json" con la codificación MacRoman.
with open("D:\HENRY\ProyectoPIMLops\PIMLops-STEAM/JSON/australian_user_reviews.json", encoding='utf-8') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))

#Crear un DataFrame de Pandas a partir de la lista de diccionarios "rows".
df_reviews = pd.DataFrame(rows)

### Revisamos nulos y vamos procesando la limpieza de datos

In [93]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [94]:
#revisamos los nulos 
nulos_por_columna = df_reviews.isnull().sum()
print(nulos_por_columna)

user_id     0
user_url    0
reviews     0
dtype: int64


***Desanidamos el diccionario de la columna "reviews"***

In [95]:
# desanidar el diccionario de la columna reviews
# cada elemento de la lista en la columna 'reviews' se convierte en una fila separada, y las demás columnas se replican en cada fila resultante.
desanidando = df_reviews.explode('reviews') 
# combinar las columnas del DataFrame desanidando con las columnas de una nueva tabla que se crea a partir de la columna 'reviews' del DataFrame desanidando.
df_user_reviews = pd.concat([desanidando.drop(['reviews'], axis=1), desanidando['reviews'].apply(pd.Series)], axis=1) 

In [96]:
df_reviews = df_user_reviews

In [97]:
df_user_reviews = df_user_reviews.drop_duplicates(subset='review', keep='first')


Eliminamos columnas irrelevantes

In [98]:
#Eliminamos la ultima columna de nuestro dataframe, que no nos sirve
df_user_reviews = df_user_reviews.iloc[:, :-1]

In [99]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55314 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      55314 non-null  object
 1   user_url     55314 non-null  object
 2   funny        55313 non-null  object
 3   posted       55313 non-null  object
 4   last_edited  55313 non-null  object
 5   item_id      55313 non-null  object
 6   helpful      55313 non-null  object
 7   recommend    55313 non-null  object
 8   review       55313 non-null  object
dtypes: object(9)
memory usage: 4.2+ MB


In [100]:
#eliminamos "funny" y "last edited"
df_user_reviews.drop(['funny', 'last_edited'], axis=1, inplace=True)

In [101]:
df_user_reviews.isna().sum()

user_id      0
user_url     0
posted       1
item_id      1
helpful      1
recommend    1
review       1
dtype: int64

In [102]:
# extraer el año de la columna 'posted' y almacenarlo en una nueva columna 'year_posted'
df_user_reviews['year_posted'] = df_user_reviews['posted'].str.extract(r'(\d{4})') 
# eliminar la columna 'posted' del DataFrame df_reviews_user
df_user_reviews.drop('posted' , axis = 1, inplace = True) 

In [103]:
df_user_reviews = df_user_reviews.rename(columns={'year_posted': 'year'})

In [104]:
#eliminamos helpful
df_user_reviews.drop(['helpful'], axis=1, inplace=True)

Observamos los value counts para calcular la media en valores faltantes de nuestra columna

In [105]:
df_user_reviews['year'].value_counts(normalize=True)

year
2014    0.449068
2015    0.376969
2013    0.136931
2012    0.024506
2011    0.011085
2010    0.001440
Name: proportion, dtype: float64

In [106]:
#Utilizamos la mediana para rellenar los datos faltantes en la columna year
df_user_reviews['year'].fillna(df_user_reviews['year'].median(), inplace=True)

In [107]:
df_user_reviews.year.isna().sum()

0

Transformamos year a int

In [108]:
df_user_reviews['year'] = df_user_reviews['year'].astype(int)

In [109]:
#Procedemos a eliminar las filas de usuarios que no hicieron ninguna reseña ni recomiendan un juego en específico
df_user_reviews = df_user_reviews.dropna(subset=['item_id', 'recommend', 'review'])

***Añadimos el análisis de sentimiento a nuestro dataset***

In [110]:
#realizamos SENTIMENT ANALYSIS
def get_sentiment(text):
    # Crear un objeto TextBlob
    blob = TextBlob(str(text))
    # Obtener el análisis de sentimiento
    sentiment = blob.sentiment.polarity
    # Devolver el análisis de sentimiento
    if sentiment < 0:
        return 0
    elif sentiment == 0:
        return 1
    else:
        return 2


aplicamos la funcion para tener el sentimiento de las reseñas

In [111]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(get_sentiment)

In [112]:
# Contar la cantidad de cada valor en la columna 'sentimiento'
conteo_sentimientos =df_user_reviews['sentiment_analysis'].value_counts()

# El resultado será una Serie de pandas con la cantidad de cada valor
print(conteo_sentimientos)

sentiment_analysis
2    31127
0    12381
1    11805
Name: count, dtype: int64


Observamos una mayor predominancia de sentimientos positivos vs neutros y negativos en todas nuestras reseñas dadas por los usuarios

In [113]:
# La función explode transforma cada elemento de una lista en una fila, replicando los valores del índice.
reviews_exploded = df_user_reviews.explode('review')
# pd.json_normalize es una función de pandas que normaliza datos semi-estructurados JSON en un dataframe plano.
rev_normalizado = pd.json_normalize(reviews_exploded['review'])
# reset_index con el parámetro drop=True descarta el índice actual y reemplaza con un índice por defecto (0, 1, 2, ...).
reviews_exploded.reset_index(drop=True, inplace=True)
rev_normalizado.reset_index(drop=True, inplace=True)
# pd.concat concatena los dataframes a lo largo de un eje. Aquí se está eliminando la columna 'review' del dataframe 'user_reviews_explored' y luego se concatena con 'user_reviews_normalized'.
df_user_reviews = pd.concat([reviews_exploded.drop('review',axis=1), rev_normalizado], axis=1)

In [114]:
df_user_reviews.isna().sum()

user_id               0
user_url              0
item_id               0
recommend             0
year                  0
sentiment_analysis    0
dtype: int64

In [115]:
df_user_reviews.fillna('SinDato',inplace=True)

***Rellenamos los valores faltantes con "SinDato" debido a que el usuario no dejó reseña y tampoco hay ID de item, se lo considera neutro***

In [116]:
df_user_reviews.isna().sum()

user_id               0
user_url              0
item_id               0
recommend             0
year                  0
sentiment_analysis    0
dtype: int64

Pasamos los "sindato" de item id a 0

In [117]:
df_user_reviews['item_id'] = df_user_reviews['item_id'].replace('SinDato', np.nan)
df_user_reviews['item_id'] = df_user_reviews['item_id'].fillna(0)

In [118]:
df_user_reviews['item_id'] = df_user_reviews['item_id'].astype(int)

In [119]:
#Generámos un codigo de conversión de tipo de dato para la columna "recommend" transformandola a booleano devolviendo True o False
def clean_data(df_user_reviews):
    df_user_reviews = df_user_reviews.astype({'recommend': 'bool'})
    return df_user_reviews

df_user_reviews = clean_data(df_user_reviews.copy())
df_user_reviews.head()

,user_id,user_url,item_id,recommend,year,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,True,2011,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,True,2011,2
3,js41637,http://steamcommunity.com/id/js41637,251610,True,2014,2
4,js41637,http://steamcommunity.com/id/js41637,227300,True,2013,0


In [120]:
#Generamos un código para la conversión de nuestro tipo de dato "user_id" a string
def clean_data(df_user_reviews):
    df_user_reviews = df_user_reviews.astype({'user_id': 'string'})
    return df_user_reviews

df_user_reviews = clean_data(df_user_reviews.copy())
df_user_reviews.head()

,user_id,user_url,item_id,recommend,year,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,True,2011,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,True,2011,2
3,js41637,http://steamcommunity.com/id/js41637,251610,True,2014,2
4,js41637,http://steamcommunity.com/id/js41637,227300,True,2013,0


In [121]:
#Generamos un código a través de la extensión Data Wrangler para la conversión de nuestro tipo de dato "user_url" a string
def clean_data(df_user_reviews):
    df_user_reviews = df_user_reviews.astype({'user_url': 'string'})
    return df_user_reviews

df_user_reviews = clean_data(df_user_reviews.copy())
df_user_reviews.head()

,user_id,user_url,item_id,recommend,year,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,True,2011,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,True,2011,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,True,2011,2
3,js41637,http://steamcommunity.com/id/js41637,251610,True,2014,2
4,js41637,http://steamcommunity.com/id/js41637,227300,True,2013,0


In [122]:
df_user_reviews = df_user_reviews.sort_values('item_id')

In [123]:
df_user_reviews

,user_id,user_url,item_id,recommend,year,sentiment_analysis
29853,Hype_R,http://steamcommunity.com/id/Hype_R,10,True,2014,2
22237,lanatbeonakeehsasamokoshtan,http://steamcommunity.com/id/lanatbeonakeehsas...,10,True,2014,2
52416,76561198089673598,http://steamcommunity.com/profiles/76561198089...,10,True,2014,1
54946,nottynotty,http://steamcommunity.com/id/nottynotty,10,True,2014,1
36294,76561198134580826,http://steamcommunity.com/profiles/76561198134...,10,True,2014,2
...,...,...,...,...,...,...
18097,76561198075141715,http://steamcommunity.com/profiles/76561198075...,521430,True,2014,0
37071,76561198071122396,http://steamcommunity.com/profiles/76561198071...,521570,True,2014,1
10360,TfhuAWGscvg,http://steamcommunity.com/id/TfhuAWGscvg,521570,True,2014,0
14618,mikeyg74,http://steamcommunity.com/id/mikeyg74,521990,True,2014,2


In [124]:
df_user_reviews

,user_id,user_url,item_id,recommend,year,sentiment_analysis
29853,Hype_R,http://steamcommunity.com/id/Hype_R,10,True,2014,2
22237,lanatbeonakeehsasamokoshtan,http://steamcommunity.com/id/lanatbeonakeehsas...,10,True,2014,2
52416,76561198089673598,http://steamcommunity.com/profiles/76561198089...,10,True,2014,1
54946,nottynotty,http://steamcommunity.com/id/nottynotty,10,True,2014,1
36294,76561198134580826,http://steamcommunity.com/profiles/76561198134...,10,True,2014,2
...,...,...,...,...,...,...
18097,76561198075141715,http://steamcommunity.com/profiles/76561198075...,521430,True,2014,0
37071,76561198071122396,http://steamcommunity.com/profiles/76561198071...,521570,True,2014,1
10360,TfhuAWGscvg,http://steamcommunity.com/id/TfhuAWGscvg,521570,True,2014,0
14618,mikeyg74,http://steamcommunity.com/id/mikeyg74,521990,True,2014,2


In [125]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55313 entries, 29853 to 13924
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             55313 non-null  string
 1   user_url            55313 non-null  string
 2   item_id             55313 non-null  int32 
 3   recommend           55313 non-null  bool  
 4   year                55313 non-null  int32 
 5   sentiment_analysis  55313 non-null  int64 
dtypes: bool(1), int32(2), int64(1), string(2)
memory usage: 2.2 MB


In [126]:
df_user_reviews.to_parquet('user_reviews.parquet')